In [ ]:
# evaluating a graph
# passing graphs for evalkuation
# techniques vs just passing 
# and how to do in ui and langsmith
# best pracices 
# E2E
# setting up evlauation process and tempaltes - repreocuidble
# all the features a framowrk and a platofrm provide!

In [ ]:
from typing import Annotated, Literal, TypedDict

from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages

import os
import nest_asyncio
from langchain.globals import set_debug, set_verbose


set_debug(True)
set_verbose(True)
nest_asyncio.apply()

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGSMITH_TRACING"] = LANGSMITH_TRACING
os.environ["LANGSMITH_ENDPOINT"] = LANGSMITH_ENDPOINT
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

class State(TypedDict): # Messages have the type "list". The 'add_messages' function # in the annotation defines how this state key should be updated # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

# Define the tools for the agent to use
@tool
def search(query: str) -> str:
    """Call to surf the web.""" # This is a placeholder, but don't tell the LLM that...
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."

tools = [search]
tool_node = ToolNode(tools)
model = init_chat_model("claude-3-5-sonnet-latest").bind_tools(tools)

# Define the function that determines whether to continue or not
def should_continue(state: State) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1] # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools" # Otherwise, we stop (reply to the user)
    return END

# Define the function that calls the model

def call_model(state: State):
    messages = state['messages']
    response = model.invoke(messages) # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define a new graph
workflow = StateGraph(State)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as 'agent'
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges( # First, we define the start node. We use 'agent'. # This means these are the edges taken after the 'agent' node is called.
    "agent", # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from 'tools' to 'agent'.
# This means that after 'tools' is called, 'agent' node is called next.
workflow.add_edge("tools", 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile()




In [103]:
from langsmith import Client

questions = [    "what's the weather in sf",
    "whats the weather in san fran",
    "whats the weather in tangier"]

answers = [    "It's 60 degrees and foggy.",
    "It's 60 degrees and foggy.",
    "It's 90 degrees and sunny.",]

ls_client = Client()

dataset = ls_client.create_dataset("weather agent", 
                                   inputs=[{"question": q} for q in questions],
                                   outputs=[{"answer": a} for a in answers])


TypeError: Client.create_dataset() got an unexpected keyword argument 'inputs'

In [ ]:
# Fixed Dataset
# Golden Dataset
# Metrics Performance
# Run Evals - Different Prompts/Models - Different from the golden set which is inputs/outputs
# Compare manually results
# Track results over time
# Automated Testing to run in CI/CD
# Eval Workflows - Langsmith
# Eval Model
# Datapoints:
# 1. Schema - Input + Maybe Output: # Expected Docs/Context Steps for agent or output - perfect answer or not 
# 2. How much - Coverage Edge Cases Guard
# 3. How gather - grow eover time - pain ponitns - synthetic - hand labelling

# evaluating what s expected in abstracted wasys

# test cases vs eval metrics

from langsmith import Client
client = Client()

dataset_name = "QA Example Dataset"
dataset = client.create_dataset(dataset_name)
# cloud eval platform but with less features

client.create_examples(dataset_id=dataset.id,
                       examples=[
                           {
            "inputs": {"question": "What is LangChain?"},
            "outputs": {"answer": "A framework for building LLM applications"},
        },
        {
            "inputs": {"question": "What is LangSmith?"},
            "outputs": {"answer": "A platform for observing and evaluating LLM applications"},
        },
        {
            "inputs": {"question": "What is OpenAI?"},
            "outputs": {"answer": "A company that creates Large Language Models"},
        },
        {
            "inputs": {"question": "What is Google?"},
            "outputs": {"answer": "A technology company known for search"},
        },
        {
            "inputs": {"question": "What is Mistral?"},
            "outputs": {"answer": "A company that creates Large Language Models"},
        }
                       ])

# Rubric schema
# eeven for strucutred outptu 
# some initial runs then eval piepeline 
# engineer must know tewstin gand eval 
# eval is for a model over a set of inputs outputs - known and unknow n cases - this is for an app


{'example_ids': ['a8d34472-29dd-4321-9c50-42a8c2f97889',
  'b90d4ebf-2758-47e6-9e74-76fd21d75593',
  '2a8b6409-8021-4302-85e9-b23d5f2be751',
  '594388d4-dc04-4430-9338-dca501b4fbb9',
  '29aa796d-1631-45f1-be54-264e395cc0b9'],
 'count': 5}

In [108]:
# type of metric requires deterministic or non deterministic evaluation in terms of measure in perfection 

from langchain_openai import ChatOpenAI
import openai
from langsmith import wrappers
from pydantic import BaseModel, Field
from sqlalchemy import desc

# eval onm prolmtp sor models

openai_client = openai.OpenAI()

eval_instructions = "You are an expert professor specialized in grading students' answers to questions."
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

class CorrectnessEvaluator(BaseModel):
    """Use this schema to structure your grading of student answers."""
    grade: Literal["CORRECT", "INCORRECT"] = Field(description="The grade of the answer, either 'CORRECT' or 'INCORRECT'.")

from langchain.prompts import ChatPromptTemplate

def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    llm_with_structured_output = llm.with_structured_output(CorrectnessEvaluator)
    prompt_template = ChatPromptTemplate.from_messages([
    ("system", eval_instructions),    
    ("human", """You are grading the following question:
    {question}
    Here is the real answer:
    {answer}
    You are grading the following predicted answer:
    {response}""")])

    correctness_runnable = prompt_template | llm_with_structured_output

    correctness_evaluator = correctness_runnable.invoke(
        {"question": inputs["question"], "response": outputs["response"], "answer": reference_outputs["answer"]})
    
    return correctness_evaluator.grade == "CORRECT"

# built in ones as well 

# evalaution resutl scoudl be classifiecation type as well
# not just binary - multi calss - so usually llm suffices this as a zero shto thign

def concision(outputs: dict, reference_outputs: dict) -> bool:
    return bool(int(len(outputs["response"]) < 2 * len(reference_outputs["answer"])))

# Test driven development
default_instructions = "Respond to the users question in a short, concise manner (one short sentence)."

def my_app(question: str, model: str = "gpt-4o-mini", instructions: str = default_instructions) -> str:
    return openai_client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": instructions},
            {"role": "user", "content": question},
        ],
    ).choices[0].message.content

# Test all aspects of our promtp etc 

def ls_target(inputs: str) -> dict:
    """The target function for LangSmith."""
    return {"response": my_app(inputs["question"])}

# We call this an experiemtn
# a promtp a dataset 
# human and metric evlaaution

experiment_results = client.evaluate(
    ls_target,
    data=dataset_name,
    evaluators=[correctness, concision], #metrics to score
    experiment_prefix="openai-4o-mini"
)

# model evaluatiosn not promtp iterations



/home/jupyter/openai-cookbook/examples/object_oriented_agentic_approach/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'openai-4o-mini-3dc20335' at:
https://smith.langchain.com/o/a5a84604-398b-52cb-a407-151c2db4c0f2/datasets/de07d102-793b-425b-a4f0-588f17de145f/compare?selectedSessions=33a58916-9fe3-411e-907e-3d66c1e86628




0it [00:00, ?it/s]

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What is OpenAI?",
  "response": "OpenAI is an artificial intelligence research organization focused on developing and promoting friendly AI.",
  "answer": "A company that creates Large Language Models"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "What is OpenAI?",
  "response": "OpenAI is an artificial intelligence research organization focused on developing and promoting friendly AI.",
  "answer": "A company that creates Large Language Models"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are an expert professor specialized in grading students' answers to questions.\nHuman: You are grading the following question:\n    What is OpenAI?\n    Here is t

1it [00:01,  1.94s/it]

...

4it [00:06,  1.50s/it]

[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\"grade\":\"CORRECT\"}",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\"grade\":\"CORRECT\"}",
            "additional_kwargs": {
              "parsed": {
                "lc": 1,
                "type": "not_implemented",
                "id": [
                  "__main__",
                  "CorrectnessEvaluator"
                ],
                "repr": "CorrectnessEvaluator(grade='CORRECT')"
              },
              "refusal": null
            },
            "response_metadata": {
              "token_usage"

5it [00:08,  1.50s/it]

[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\"grade\":\"INCORRECT\"}",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\"grade\":\"INCORRECT\"}",
            "additional_kwargs": {
              "parsed": {
                "lc": 1,
                "type": "not_implemented",
                "id": [
                  "__main__",
                  "CorrectnessEvaluator"
                ],
                "repr": "CorrectnessEvaluator(grade='INCORRECT')"
              },
              "refusal": null
            },
            "response_metadata": {
              "token_

5it [00:08,  1.67s/it]


In [109]:

def ls_target_v2(inputs: str) -> dict:
    return {"response": my_app(inputs["question"], model="gpt-4-turbo")}

experiment_results = client.evaluate(
    ls_target_v2,
    data=dataset_name,
    evaluators=[concision, correctness],
    experiment_prefix="openai-4-turbo",
)

View the evaluation results for experiment: 'openai-4-turbo-c461d936' at:
https://smith.langchain.com/o/a5a84604-398b-52cb-a407-151c2db4c0f2/datasets/de07d102-793b-425b-a4f0-588f17de145f/compare?selectedSessions=449028e6-7b54-4f15-ad21-57660e592161




0it [00:00, ?it/s]

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What is OpenAI?",
  "response": "OpenAI is an artificial intelligence research lab that focuses on developing and promoting friendly AI in a way that benefits humanity as a whole.",
  "answer": "A company that creates Large Language Models"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "What is OpenAI?",
  "response": "OpenAI is an artificial intelligence research lab that focuses on developing and promoting friendly AI in a way that benefits humanity as a whole.",
  "answer": "A company that creates Large Language Models"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are an expert professor specialized in grading students' answers to questions.\nHuma

1it [00:01,  1.64s/it]

[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\"grade\":\"CORRECT\"}",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\"grade\":\"CORRECT\"}",
            "additional_kwargs": {
              "parsed": {
                "lc": 1,
                "type": "not_implemented",
                "id": [
                  "__main__",
                  "CorrectnessEvaluator"
                ],
                "repr": "CorrectnessEvaluator(grade='CORRECT')"
              },
              "refusal": null
            },
            "response_metadata": {
              "token_usage"

...

4it [00:07,  2.18s/it]

[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.54s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\"grade\":\"CORRECT\"}",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\"grade\":\"CORRECT\"}",
            "additional_kwargs": {
              "parsed": {
                "lc": 1,
                "type": "not_implemented",
                "id": [
                  "__main__",
                  "CorrectnessEvaluator"
                ],
                "repr": "CorrectnessEvaluator(grade='CORRECT')"
              },
              "refusal": null
            },
            "response_metadata": {
              "token_u

5it [00:09,  1.91s/it]

[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\"grade\":\"INCORRECT\"}",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\"grade\":\"INCORRECT\"}",
            "additional_kwargs": {
              "parsed": {
                "lc": 1,
                "type": "not_implemented",
                "id": [
                  "__main__",
                  "CorrectnessEvaluator"
                ],
                "repr": "CorrectnessEvaluator(grade='INCORRECT')"
              },
              "refusal": null
            },
            "response_metadata": {
              "token_

5it [00:09,  1.93s/it]


In [111]:
instructions_v3 = "Respond to the users question in a short, concise manner (one short sentence). Do NOT use more than ten words."

def ls_target_v3(inputs: str) -> dict:
    response = my_app(
        inputs["question"], 
        model="gpt-4-turbo",
        instructions=instructions_v3
    )
    return {"response": response}

# experiemtn is linked to a datset - wehre you change the parameters of the models used and the features or promtps
# baseline
# very importnat to iterate like this - baseline run for a n expeirment and for a a metric
# can be xacross experiements
# chanign the input outpuit format means its a differnt experment and not comaprabel anymroe to the any behcmark sets as well


experiment_results = client.evaluate(
    ls_target_v3,
    data=dataset_name,
    evaluators=[concision, correctness],
    experiment_prefix="strict-openai-4-turbo",
)

# lookign for improvements aor regressions
# 3 MANUAL COMAPRISON

# CRITERAI OF PASS FROM THE RESUTLS from a baseline
# over an experiemtn - over a datset not jsut one - many runs - could even do repateabilty fi temrparture is chanign

# test is what says if we pass or not criterai

# evalautors - human or metric based - they are able to let you score to create test cases 

# for non llm apps its not a meteric its a simple eval often so thats why its easier
# edge cases etc coverage
# coverage of edge cases vs if criteiria passes
# tests are asserts - in programming can assert weach outptu 
# hear edo it over larger numebr?
# test team shoudl set this up 
# only for dev team for test driven dev unti test ans dudnerstanding 

# to even design my own should know what comomon framowrks have 
# sairam
# to make nintex specific
# sairam

def test_length_score() -> None:
    "Test that the length of the score is at least 80%"
    experiment_results = client.evaluate(ls_target, data=dataset_name, evaluators=[concision, correctness])
    scores = [result.score for result in experiment_results.results]
    assert sum(scores) / len(scores) >= 0.8, "The average score is less than 80%"

View the evaluation results for experiment: 'strict-openai-4-turbo-b2260485' at:
https://smith.langchain.com/o/a5a84604-398b-52cb-a407-151c2db4c0f2/datasets/de07d102-793b-425b-a4f0-588f17de145f/compare?selectedSessions=0413ee9a-3086-4925-8a81-a77a0aadf022




0it [00:00, ?it/s]

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What is OpenAI?",
  "response": "OpenAI is an AI research and deployment company.",
  "answer": "A company that creates Large Language Models"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "What is OpenAI?",
  "response": "OpenAI is an AI research and deployment company.",
  "answer": "A company that creates Large Language Models"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are an expert professor specialized in grading students' answers to questions.\nHuman: You are grading the following question:\n    What is OpenAI?\n    Here is the real answer:\n    A company that creates Large Language Models\n    You are grading the following predicted answer:

1it [00:02,  2.38s/it]

[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\"grade\":\"CORRECT\"}",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\"grade\":\"CORRECT\"}",
            "additional_kwargs": {
              "parsed": {
                "lc": 1,
                "type": "not_implemented",
                "id": [
                  "__main__",
                  "CorrectnessEvaluator"
                ],
                "repr": "CorrectnessEvaluator(grade='CORRECT')"
              },
              "refusal": null
            },
            "response_metadata": {
              "token_usage"

...

4it [00:09,  2.56s/it]

[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [2.64s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\"grade\":\"CORRECT\"}",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\"grade\":\"CORRECT\"}",
            "additional_kwargs": {
              "parsed": {
                "lc": 1,
                "type": "not_implemented",
                "id": [
                  "__main__",
                  "CorrectnessEvaluator"
                ],
                "repr": "CorrectnessEvaluator(grade='CORRECT')"
              },
              "refusal": null
            },
            "response_metadata": {
              "token_u

5it [00:11,  2.15s/it]

[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\"grade\":\"INCORRECT\"}",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\"grade\":\"INCORRECT\"}",
            "additional_kwargs": {
              "parsed": {
                "lc": 1,
                "type": "not_implemented",
                "id": [
                  "__main__",
                  "CorrectnessEvaluator"
                ],
                "repr": "CorrectnessEvaluator(grade='INCORRECT')"
              },
              "refusal": null
            },
            "response_metadata": {
              "token_

5it [00:11,  2.31s/it]


In [112]:
test_length_score()

View the evaluation results for experiment: 'kind-number-27' at:
https://smith.langchain.com/o/a5a84604-398b-52cb-a407-151c2db4c0f2/datasets/de07d102-793b-425b-a4f0-588f17de145f/compare?selectedSessions=1a4f6a62-01f9-44dc-9bb0-cd604abd37dc




0it [00:00, ?it/s]

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What is OpenAI?",
  "response": "OpenAI is an artificial intelligence research organization focused on developing and promoting friendly AI.",
  "answer": "A company that creates Large Language Models"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "What is OpenAI?",
  "response": "OpenAI is an artificial intelligence research organization focused on developing and promoting friendly AI.",
  "answer": "A company that creates Large Language Models"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are an expert professor specialized in grading students' answers to questions.\nHuman: You are grading the following question:\n    What is OpenAI?\n    Here is t

1it [00:01,  1.20s/it]

[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\"grade\":\"CORRECT\"}",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\"grade\":\"CORRECT\"}",
            "additional_kwargs": {
              "parsed": {
                "lc": 1,
                "type": "not_implemented",
                "id": [
                  "__main__",
                  "CorrectnessEvaluator"
                ],
                "repr": "CorrectnessEvaluator(grade='CORRECT')"
              },
              "refusal": null
            },
            "response_metadata": {
              "token_usage"

...

[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\"grade\":\"CORRECT\"}",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\"grade\":\"CORRECT\"}",
            "additional_kwargs": {
              "parsed": {
                "lc": 1,
                "type": "not_implemented",
                "id": [
                  "__main__",
                  "CorrectnessEvaluator"
                ],
                "repr": "CorrectnessEvaluator(grade='CORRECT')"
              },
              "refusal": null
            },
            "response_metadata": {
              "token_usage"

5it [00:05,  1.24s/it]

[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\"grade\":\"INCORRECT\"}",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\"grade\":\"INCORRECT\"}",
            "additional_kwargs": {
              "parsed": {
                "lc": 1,
                "type": "not_implemented",
                "id": [
                  "__main__",
                  "CorrectnessEvaluator"
                ],
                "repr": "CorrectnessEvaluator(grade='INCORRECT')"
              },
              "refusal": null
            },
            "response_metadata": {
              "token_

5it [00:06,  1.23s/it]


AttributeError: 'ExperimentResults' object has no attribute 'results'

In [ ]:
# NEW PROJECT BELOW

In [ ]:
# First, import the required packages
import os
from dotenv import load_dotenv
import nest_asyncio
from langchain.globals import set_debug, set_verbose

# Load environment variables from .env file
load_dotenv()

# Set debug and verbose mode
set_debug(True)
set_verbose(True)
nest_asyncio.apply()

# Get environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGSMITH_TRACING = os.getenv("LANGSMITH_TRACING")
LANGSMITH_ENDPOINT = os.getenv("LANGSMITH_ENDPOINT")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# Set environment variables
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGSMITH_TRACING"] = LANGSMITH_TRACING
os.environ["LANGSMITH_ENDPOINT"] = LANGSMITH_ENDPOINT
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

In [113]:
# backtesting vs particualr datsets
# new datset iwth prod data
# not all evals on reference data 
# here back testing is a way to test
# feedback is a form of gropujnd truth 

# use case of naviagting a digital music store - so enterpreise co pilots or chat bots
# baisclaly they are digital apps avialabel which ewe enhancing with bots to do things autoamtically on users behlf and with naturla lkangauge interaface

# final resposne trakaecotry each individual step
import requests

url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"

response = requests.get(url)

if response.status_code == 200:
    # Open a local file in binary write mode
    with open("chinook.db", "wb") as file:
        # Write the content of the response (the file) to the local file
        file.write(response.content)
    print("File downloaded and saved as Chinook.db")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")


File downloaded and saved as Chinook.db


In [114]:
# itneracitosn for agent
# mock
# query paremters determiend by llm vs given by user or from chat 
# dict and float

import sqlite3

@tool
def _refund(invoice_id: int | None, invoice_line_ids: list[int] | None, mock: bool = False) -> float:
    """Given an Invoice ID and/or Invoice Line IDs, delete the relevant Invoice/InvoiceLine records in the Chinook DB.

    Args:
        invoice_id: The Invoice to delete.
        invoice_line_ids: The Invoice Lines to delete.
        mock: If True, do not actually delete the specified Invoice/Invoice Lines. Used for testing purposes.

    Returns:
        float: The total dollar amount that was deleted (or mock deleted).
    """

    if invoice_id is None and invoice_line_ids is None:
        return 0.0

    # Connect to the Chinook database
    conn = sqlite3.connect("chinook.db")
    cursor = conn.cursor()

    total_refund = 0.0

    try:
        # If invoice_id is provided, delete entire invoice and its lines
        if invoice_id is not None:
            # First get the total amount for the invoice
            cursor.execute(
                """
                SELECT Total
                FROM Invoice
                WHERE InvoiceId = ?
            """,
                (invoice_id,),
            )

            result = cursor.fetchone()
            if result:
                total_refund += result[0]

            # Delete invoice lines first (due to foreign key constraints)
            if not mock:
                cursor.execute(
                    """
                    DELETE FROM InvoiceLine
                    WHERE InvoiceId = ?
                """,
                    (invoice_id,),
                )

                # Then delete the invoice
                cursor.execute(
                    """
                    DELETE FROM Invoice
                    WHERE InvoiceId = ?
                """,
                    (invoice_id,),
                )

        # If specific invoice lines are provided
        if invoice_line_ids is not None:
            # Get the total amount for the specified invoice lines
            placeholders = ",".join(["?" for _ in invoice_line_ids])
            cursor.execute(
                f"""
                SELECT SUM(UnitPrice * Quantity)
                FROM InvoiceLine
                WHERE InvoiceLineId IN ({placeholders})
            """,
                invoice_line_ids,
            )

            result = cursor.fetchone()
            if result and result[0]:
                total_refund += result[0]

            if not mock:
                # Delete the specified invoice lines
                cursor.execute(
                    f"""
                    DELETE FROM InvoiceLine
                    WHERE InvoiceLineId IN ({placeholders})
                """,
                    invoice_line_ids,
                )

        # Commit the changes
        conn.commit()

    except sqlite3.Error as e:
        # Roll back in case of error
        conn.rollback()
        raise e

    finally:
        # Close the connection
        conn.close()

    return float(total_refund)

@tool
def _lookup(
    customer_first_name: str,
    customer_last_name: str,
    customer_phone: str,
    track_name: str | None,
    album_title: str | None,
    artist_name: str | None,
    purchase_date_iso_8601: str | None,
) -> list[dict]:
    """Find all of the Invoice Line IDs in the Chinook DB for the given filters.

    Returns:
        a list of dictionaries that contain keys: {
            'invoice_line_id',
            'track_name',
            'artist_name',
            'purchase_date',
            'quantity_purchased',
            'price_per_unit'
        }
    """

    # Connect to the database
    conn = sqlite3.connect("chinook.db")
    cursor = conn.cursor()

    # Base query joining all necessary tables
    query = """
    SELECT
        il.InvoiceLineId,
        t.Name as track_name,
        art.Name as artist_name,
        i.InvoiceDate as purchase_date,
        il.Quantity as quantity_purchased,
        il.UnitPrice as price_per_unit
    FROM InvoiceLine il
    JOIN Invoice i ON il.InvoiceId = i.InvoiceId
    JOIN Customer c ON i.CustomerId = c.CustomerId
    JOIN Track t ON il.TrackId = t.TrackId
    JOIN Album alb ON t.AlbumId = alb.AlbumId
    JOIN Artist art ON alb.ArtistId = art.ArtistId
    WHERE c.FirstName = ?
    AND c.LastName = ?
    AND c.Phone = ?
    """

    # Parameters for the query
    params = [customer_first_name, customer_last_name, customer_phone]

    # Add optional filters
    if track_name:
        query += " AND t.Name = ?"
        params.append(track_name)

    if album_title:
        query += " AND alb.Title = ?"
        params.append(album_title)

    if artist_name:
        query += " AND art.Name = ?"
        params.append(artist_name)

    if purchase_date_iso_8601:
        query += " AND date(i.InvoiceDate) = date(?)"
        params.append(purchase_date_iso_8601)

    # Execute query
    cursor.execute(query, params)

    # Fetch results
    results = cursor.fetchall()

    # Convert results to list of dictionaries
    output = []
    for row in results:
        output.append(
            {
                "invoice_line_id": row[0],
                "track_name": row[1],
                "artist_name": row[2],
                "purchase_date": row[3],
                "quantity_purchased": row[4],
                "price_per_unit": row[5],
            }
        )

    # Close connection
    conn.close()

    return output

In [ ]:
# Extrraction in cusotmer suppoer tusually first node - not coming from config etc or get memory from there
# route to paths to workflwo siwth on tools and prompts
# threee paths - this routign changes in mutli agent - not just skill based workflows and agent
# but dlegation and coollaboration
# with messages queue
# build more 
# sufficient detaisl check - promtp to do that 
# If we can extract info based on intent
# and tell them what they dont have etc 
# Resposne Path 
# Maybe dont do fucniton call just have output strucutred then pass to tool node 
# extracted info 
# response
# keep track of produc tinfo etc in the state not jsut messges so that its easy to take it out from state
# extracting info in system message or from state have a tool to get it 

In [ ]:
# ############### OLD APPROACH
# from typing import TypedDict, Union, NotRequired
# class InvoiceIdOnly(TypedDict):
#     """Mandatory fields for refunding an invoice."""
#     invoice_id: int = Field(description="The ID of the invoice to refund.")
#     invoice_line_ids: NotRequired[list[int]] = Field(
#         description="A list of Invoice Line IDs to refund. If provided, the invoice will be refunded for these specific lines only."
#     )

# class InvoiceLineIdsOnly(TypedDict):
#     invoice_id: NotRequired[int] = Field(
#         description="The ID of the invoice to refund."
#     )
#     invoice_line_ids: list[int] = Field(
#         description="A list of Invoice Line IDs to refund. If provided, the invoice will be refunded for these specific lines only."
#     )

# class BothFields(TypedDict):
#     invoice_id: int = Field(description="The ID of the invoice to refund.")
#     invoice_line_ids: list[int] = Field(
#         description="A list of Invoice Line IDs to refund. If provided, the invoice will be refunded for these specific lines only."
#     )

# RefundMandatoryFields = Union[InvoiceIdOnly, InvoiceLineIdsOnly, BothFields]

# class LookupMandatoryFields(TypedDict):
#     customer_first_name: str = Field(description="The first name of the customer.")
#     customer_last_name: str = Field(description="The last name of the customer.")
#     customer_phone: str = Field(description="The phone number of the customer.")

# class TextResponse(TypedDict):
#     """Response from the Router Agent."""
#     response: str

# class RouterAgentResponse(TypedDict):
#     """Response from the Router Agent."""
#     user_intent: str
#     extracted_fields_or_response: Union[RefundMandatoryFields, LookupMandatoryFields, TextResponse]

# workflow.add_node("capture_intent")
# workflow.add_node("extract_info", extract_info)

####################################

from langgraph.graph.message import MessagesState
from langgraph.prebuilt import tools_condition

class State(MessagesState):
    input: str

def router(state: State) -> State:
    system_prompt_tempalte = """You are an expert customer support router agent. 
    You are the entry point to the customer support system.
    You interact with the user to understand their intent, extract relevant information and fulfill the request.
    Your task is to determine the user's intent, extract the necessary information based on that intent, call tools to gather information and respond back to the user.
    You have access to the following tools, which represent agents that have have those specific skills and can execute those workflows:
    1. Refund Tool: Use this tool to refund an invoice or specific invoice lines.
    2. Lookup Tool: Use this tool to look up invoice lines based on customer information.
    Use a tool if you have enough of the Required Fields of that tool to use it, extracted from the user input and if the intent of the user is to perform that particular function. Both must be present for you to be able to use the tool.
    If the user intent suggests using a tool, but you do not have enough mandatory information, respond back to the user specifying what information is missing.
    If the user intent is outside the skill sets which you can route to, respond with a message that you cannot help with that. And that only specific requests are possible.
    Do not assist in any other way that is outside your skillset of using these tools, responding back regarding those tools if intent matches, or indicating that you do not take up any other requests.
    If you have enough information to fulfill the request, respond back to the user answering their question or confirming that you have executed the request."""

    tools = [_refund, _lookup]

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

    llm_with_tools = llm.bind_tools(tools)

    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt_tempalte),
        ("user", "{input}"),
    ])

    router_runnable = prompt_template | llm_with_tools

    message = router_runnable.invoke({"input": state["input"]})

    return {"messages": [message]}

# def response_node(state: State) -> State:

workflow = StateGraph(State)
workflow.add_node("router", router)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "router")
workflow.add_edge("router", END)
workflow.add_conditional_edges(
    "router",
    tools_condition,
    {
        "tools": "tools",
        END: END,
    },
)
workflow.add_edge("tools", "router")

config = {"configurable": {"thread_id": 1}}
checkpointer = Checkpointer(workflow, config=config)

workflow.compile(checkpointer=Checkpointer)


# nto really an agent - more like a workflow - why doi we need extracted tool call info 
# to store separately 
# react agent vs workflows becuase we know the paths well?
# put the heavy liftign to the pormpt not to our design as that makes it very hard
# more tesitn  can be done with evalautors but less dynamic and becoems workflow like - hard to maintain
# only when you know the predicatbilty very well - architect the system well in that case
# maybe start alwys deifning promtp tepalte
# messages for sceanriso that build up the state in list in more than oen tthread 
# dotn ahve lsits
# no strucutred output 
# need guardraisls
# store the tools fields so can use or use memory to do that langmem etc 
# authrosiation and authentication etc 
# do not have to ahve intor messge on the message hist - thats input side - outside app
# must see ienoguh apps as well to gain intutiuioon of what needed on app side
# need to build a lot of apps to get the intuition of what is needed
# threades of takign user inpout or succive calls are not really agentic or with human in the loop
# ro interrupts for tools can use btu thats separte part of agent 
# those toerhs are seprate thresads and memory must be used in those scnearios
# buidl this out and evaluate it 
# see all I can do 
# started witha nd idea now wwant to change it is abosltuely fine 
# dotn get frsutrated to change idea and dont always look at the time
# keep going 
# add bells whistles includign memeory and try evalaute 
# sairam
# simple agent 
# then can scale agent
# sairam
# steps can be combined into rotuers etc when we start noticing power of promtps
# add all I learnt to a project
# sairam
# do it each day 
# sairam
# learn somethign new each day 
# dont do refactor not so improtnat stuff now - later - now BUILD BUILD BUILD - SHIP SHIP SHIP - SHARE SHARE SHARE - SAIRAM